In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime

In [2]:
target_gu = '구로구'
target_dong = '구로동'
#target_purpose = '준주거'
target_building_age = 3
target_period = 18 # in months

In [3]:
date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')

In [4]:
date_today_record = str(date_today).split(' ')[0].replace('-', '')
date_today_record

'20210623'

In [5]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [6]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [7]:
def data_prep(bdtype, tradetype):
    # bdtype is a string: one of 아파트, 연립다세대 or 오피스텔
    # tradetype is a string: one of 매매 or 전월세
    
    basedir = './국토교통부_실거래가_공개시스템/{}/{}/'.format(bdtype, tradetype)
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('(' in f)]
    
    dfs_list = []
    for i, f in tqdm(enumerate(filenames)):
        df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        if '해제사유발생일' in df.columns.tolist():
            df = df[df['해제사유발생일'].isna()]
            df = df.drop(columns=['해제사유발생일'])

        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
        
    #concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    #concat_df['계약날짜기준_건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    #concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
        
    cols_to_drop = ['계약년월', '계약일', '도로명']
        
    concat_df = concat_df[[col for col in concat_df.columns if col not in cols_to_drop]]
    
    date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')
    #concat_df['건물연식'] = date_today.year - concat_df['건축년도']
    
    #concat_df = concat_df.dropna(subset=['지번주소'])
    
    if '건물명' in concat_df.columns:
        concat_df.rename(columns={'건물명':'건물(단지)명'}, inplace=True)
    elif '단지명' in concat_df.columns:
        concat_df.rename(columns={'단지명':'건물(단지)명'}, inplace=True)
    
    concat_df = concat_df[concat_df['층'] >= 0].reset_index(drop=True)
    
    return concat_df

In [8]:
yunrip_df = data_prep('연립다세대', '매매')
yunrip_df['부동산유형'] = '연립다세대'
yunrip_df['지번주소'] = yunrip_df['시군구'] + ' ' + yunrip_df['번지']
print(yunrip_df.shape)
yunrip_df.head()

16it [00:01, 10.52it/s]


(639674, 14)


,시군구,번지,본번,부번,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,지번주소
0,서울특별시 강남구 개포동,171-13,171,13,(171-13),68.08,56.10,56500,2,1988.0,2006-12-20,829.905993,연립다세대,서울특별시 강남구 개포동 171-13
1,서울특별시 강남구 개포동,1239-7,1239,7,강남빌라 가동,52.59,45.00,40500,2,1988.0,2006-12-09,770.108386,연립다세대,서울특별시 강남구 개포동 1239-7
2,서울특별시 강남구 개포동,1239-7,1239,7,강남빌라 가동,52.59,45.00,42000,1,1988.0,2006-12-19,798.630918,연립다세대,서울특별시 강남구 개포동 1239-7
3,서울특별시 강남구 개포동,1239-6,1239,6,강남빌라 나동,52.75,44.68,22000,2,1988.0,2006-11-02,417.061611,연립다세대,서울특별시 강남구 개포동 1239-6
4,서울특별시 강남구 개포동,1175-2,1175,2,강남파크,49.43,23.70,17000,4,1993.0,2006-07-19,343.920696,연립다세대,서울특별시 강남구 개포동 1175-2


In [10]:
yunrip_df.isna().sum()

시군구          0
번지          47
본번           0
부번           0
건물(단지)명      0
전용면적(㎡)      0
대지권면적(㎡)     0
거래금액(만원)     0
층            0
건축년도        25
계약날짜         0
단가(만원/㎡)     0
부동산유형        0
지번주소        47
dtype: int64

In [11]:
yunrip_df.dropna(subset=['번지'], inplace=True)

In [12]:
inv_df = yunrip_df[yunrip_df['번지'].str.contains('외')]
print(inv_df.shape)
inv_df.head()

(0, 14)


,시군구,번지,본번,부번,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,지번주소


In [13]:
yunrip_target_df = yunrip_df[
    (yunrip_df['시군구'].str.contains('구로구 구로'))
    &(yunrip_df['계약날짜'] >= (date_today - pd.DateOffset(months=target_period)))
].sort_values(['시군구', '본번', '부번', '계약날짜']).reset_index(drop=True)
print(yunrip_target_df.shape)
yunrip_target_df.head()

(634, 14)


,시군구,번지,본번,부번,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,지번주소
0,서울특별시 구로구 구로동,3-81,3,81,한영선샤인빌리지,44.40,20.30,37500,5,2017.0,2020-11-21,844.594595,연립다세대,서울특별시 구로구 구로동 3-81
1,서울특별시 구로구 구로동,3-81,3,81,한영선샤인빌리지,42.73,19.53,47800,4,2017.0,2021-01-09,1118.652001,연립다세대,서울특별시 구로구 구로동 3-81
2,서울특별시 구로구 구로동,3-81,3,81,한영선샤인빌리지,38.54,17.62,35500,4,2017.0,2021-05-07,921.120913,연립다세대,서울특별시 구로구 구로동 3-81
3,서울특별시 구로구 구로동,3-81,3,81,한영선샤인빌리지,38.54,17.62,35500,3,2017.0,2021-05-23,921.120913,연립다세대,서울특별시 구로구 구로동 3-81
4,서울특별시 구로구 구로동,23-4,23,4,(23-4),36.36,29.11,18000,3,1988.0,2020-07-04,495.049505,연립다세대,서울특별시 구로구 구로동 23-4


In [15]:
land_specs_df = pd.read_csv('./prepped_data/land_specs_baseline.csv')
print(land_specs_df.shape)
land_specs_df.head()

(8405705, 10)


,지번주소,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 개포동 100-0,2013.0,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
1,서울특별시 강남구 개포동 100-0,2014.0,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
2,서울특별시 강남구 개포동 100-0,2015.0,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
3,서울특별시 강남구 개포동 100-0,2016.0,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
4,서울특별시 강남구 개포동 100-0,2017.0,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지


In [16]:
last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년']).reset_index(drop=True)
last_df['지번주소'] = last_df['지번주소'].apply(lambda x: x.replace('-0', ''))
print(last_df.shape)
last_df.head()

(982961, 9)


,지번주소,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 개포동 100,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
1,서울특별시 강남구 개포동 101,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지
2,서울특별시 강남구 개포동 102,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지
3,서울특별시 강남구 개포동 103,전,516.0,개발제한구역,자연녹지지역,전,평지,부정형,맹지
4,서울특별시 강남구 개포동 104-1,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지


In [17]:
last_df['도로접면'].value_counts()

세로한면(가)    310836
지정되지않음     154016
세로한면(불)    120498
세로각지(가)     81207
소로한면        71216
소로각지        54559
광대소각        36761
중로각지        34874
중로한면        33846
광대로한면       31789
맹지          22147
광대세각        18110
세로각지(불)     12789
Name: 도로접면, dtype: int64

In [18]:
yunrip_merge_df = yunrip_target_df.merge(last_df, on=['지번주소'])
print(yunrip_merge_df.shape)
yunrip_merge_df.head()

(634, 22)


,시군구,번지,본번,부번,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,지번주소,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 구로구 구로동,3-81,3,81,한영선샤인빌리지,44.40,20.30,37500,5,2017.0,2020-11-21,844.594595,연립다세대,서울특별시 구로구 구로동 3-81,종교용지,795.0,제3종일반주거지역,지정되지않음,주거기타,평지,사다리형,광대소각
1,서울특별시 구로구 구로동,3-81,3,81,한영선샤인빌리지,42.73,19.53,47800,4,2017.0,2021-01-09,1118.652001,연립다세대,서울특별시 구로구 구로동 3-81,종교용지,795.0,제3종일반주거지역,지정되지않음,주거기타,평지,사다리형,광대소각
2,서울특별시 구로구 구로동,3-81,3,81,한영선샤인빌리지,38.54,17.62,35500,4,2017.0,2021-05-07,921.120913,연립다세대,서울특별시 구로구 구로동 3-81,종교용지,795.0,제3종일반주거지역,지정되지않음,주거기타,평지,사다리형,광대소각
3,서울특별시 구로구 구로동,3-81,3,81,한영선샤인빌리지,38.54,17.62,35500,3,2017.0,2021-05-23,921.120913,연립다세대,서울특별시 구로구 구로동 3-81,종교용지,795.0,제3종일반주거지역,지정되지않음,주거기타,평지,사다리형,광대소각
4,서울특별시 구로구 구로동,23-4,23,4,(23-4),36.36,29.11,18000,3,1988.0,2020-07-04,495.049505,연립다세대,서울특별시 구로구 구로동 23-4,대,232.9,제3종일반주거지역,지정되지않음,다세대,평지,가로장방,세로한면(가)


In [47]:
inv_df = yunrip_merge_df[
    (yunrip_merge_df['전용면적(㎡)'] >= 28)
    &(yunrip_merge_df['전용면적(㎡)'] <= 32)
]
inv_df = inv_df[inv_df['건축년도'] >= 2021].sort_values(['단가(만원/㎡)'], ascending=False)
print(inv_df.shape)
inv_df

(9, 22)


,시군구,번지,본번,부번,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,지번주소,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
138,서울특별시 구로구 구로동,317-165,317,165,제이펠리체,31.89,22.36,42800,5,2021.0,2021-04-29,1342.113515,연립다세대,서울특별시 구로구 구로동 317-165,대,172.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)
197,서울특별시 구로구 구로동,429-154,429,154,모드하우스,29.98,19.90,26550,4,2021.0,2021-06-18,885.590394,연립다세대,서울특별시 구로구 구로동 429-154,대,225.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)
190,서울특별시 구로구 구로동,429-154,429,154,모드하우스,29.98,19.90,25800,4,2021.0,2021-06-18,860.573716,연립다세대,서울특별시 구로구 구로동 429-154,대,225.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)
196,서울특별시 구로구 구로동,429-154,429,154,모드하우스,29.15,19.35,23800,5,2021.0,2021-06-18,816.466552,연립다세대,서울특별시 구로구 구로동 429-154,대,225.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)
198,서울특별시 구로구 구로동,429-154,429,154,모드하우스,29.03,19.27,23450,5,2021.0,2021-06-18,807.785050,연립다세대,서울특별시 구로구 구로동 429-154,대,225.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)
192,서울특별시 구로구 구로동,429-154,429,154,모드하우스,29.91,19.85,24050,3,2021.0,2021-06-18,804.078903,연립다세대,서울특별시 구로구 구로동 429-154,대,225.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)
191,서울특별시 구로구 구로동,429-154,429,154,모드하우스,29.25,19.41,23500,3,2021.0,2021-06-18,803.418803,연립다세대,서울특별시 구로구 구로동 429-154,대,225.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)
194,서울특별시 구로구 구로동,429-154,429,154,모드하우스,29.91,19.85,23800,2,2021.0,2021-06-18,795.720495,연립다세대,서울특별시 구로구 구로동 429-154,대,225.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)
199,서울특별시 구로구 구로동,429-154,429,154,모드하우스,29.25,19.41,23250,2,2021.0,2021-06-18,794.871795,연립다세대,서울특별시 구로구 구로동 429-154,대,225.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)


In [49]:
inv_df['단가(만원/㎡)'].mean()

878.95769148488

In [27]:
yunrip_merge_df = yunrip_merge_df.sort_values(['단가(만원/㎡)'], ascending=False)

In [29]:
yunrip_merge_df.head(50)

,시군구,번지,본번,부번,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,지번주소,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
349,서울특별시 구로구 구로동,706-1,706,1,베스트빌1차,19.84,14.10,26800,4,2020.0,2021-05-20,1350.806452,연립다세대,서울특별시 구로구 구로동 706-1,대,460.0,제2종일반주거지역,지정되지않음,상업용,평지,부정형,소로각지
138,서울특별시 구로구 구로동,317-165,317,165,제이펠리체,31.89,22.36,42800,5,2021.0,2021-04-29,1342.113515,연립다세대,서울특별시 구로구 구로동 317-165,대,172.0,제2종일반주거지역,지정되지않음,단독,완경사,정방형,세로한면(가)
344,서울특별시 구로구 구로동,706-1,706,1,베스트빌1차,22.80,16.70,27800,4,2020.0,2021-04-30,1219.298246,연립다세대,서울특별시 구로구 구로동 706-1,대,460.0,제2종일반주거지역,지정되지않음,상업용,평지,부정형,소로각지
346,서울특별시 구로구 구로동,706-1,706,1,베스트빌1차,22.80,16.70,27300,3,2020.0,2021-05-06,1197.368421,연립다세대,서울특별시 구로구 구로동 706-1,대,460.0,제2종일반주거지역,지정되지않음,상업용,평지,부정형,소로각지
345,서울특별시 구로구 구로동,706-1,706,1,베스트빌1차,22.80,16.70,27200,5,2020.0,2021-05-06,1192.982456,연립다세대,서울특별시 구로구 구로동 706-1,대,460.0,제2종일반주거지역,지정되지않음,상업용,평지,부정형,소로각지
347,서울특별시 구로구 구로동,706-1,706,1,베스트빌1차,23.44,16.70,27400,5,2020.0,2021-05-07,1168.941980,연립다세대,서울특별시 구로구 구로동 706-1,대,460.0,제2종일반주거지역,지정되지않음,상업용,평지,부정형,소로각지
348,서울특별시 구로구 구로동,706-1,706,1,베스트빌1차,23.44,16.70,27400,5,2020.0,2021-05-07,1168.941980,연립다세대,서울특별시 구로구 구로동 706-1,대,460.0,제2종일반주거지역,지정되지않음,상업용,평지,부정형,소로각지
254,서울특별시 구로구 구로동,466-299,466,299,베네치아,30.00,18.86,34000,2,2019.0,2020-07-14,1133.333333,연립다세대,서울특별시 구로구 구로동 466-299,대,119.0,제2종일반주거지역,지정되지않음,단독,완경사,자루형,세로한면(가)
1,서울특별시 구로구 구로동,3-81,3,81,한영선샤인빌리지,42.73,19.53,47800,4,2017.0,2021-01-09,1118.652001,연립다세대,서울특별시 구로구 구로동 3-81,종교용지,795.0,제3종일반주거지역,지정되지않음,주거기타,평지,사다리형,광대소각
301,서울특별시 구로구 구로동,551-133,551,133,대흥아트빌,44.06,26.48,48000,6,2012.0,2021-03-29,1089.423513,연립다세대,서울특별시 구로구 구로동 551-133,대,318.0,제2종일반주거지역,지정되지않음,주거나지,완경사,사다리형,세로한면(가)


In [22]:
inv_df = last_df[last_df['지번주소'].str.contains('구로동 481-6')]
print(inv_df.shape)
inv_df

(1, 9)


,지번주소,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
253250,서울특별시 구로구 구로동 481-6,대,660.0,제3종일반주거지역,지정되지않음,상업용,평지,사다리형,중로각지


In [23]:
yunrip_target_df = yunrip_merge_df[yunrip_merge_df['도로접면'].str.contains('중로')]
print(yunrip_target_df.shape)
yunrip_target_df.head()

(39, 22)


,시군구,번지,본번,부번,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,지번주소,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
6,서울특별시 구로구 구로동,33,33,0,우성빌라,55.06,70.94,39500,1,1985.0,2019-12-30,717.399201,연립다세대,서울특별시 구로구 구로동 33,대,1981.0,제2종일반주거지역,지정되지않음,연립,평지,가로장방,중로각지
7,서울특별시 구로구 구로동,33,33,0,우성빌라,55.06,70.94,40000,2,1985.0,2020-02-11,726.480203,연립다세대,서울특별시 구로구 구로동 33,대,1981.0,제2종일반주거지역,지정되지않음,연립,평지,가로장방,중로각지
35,서울특별시 구로구 구로동,111,111,0,대원파크빌,43.88,20.76,28800,4,2016.0,2020-10-13,656.335460,연립다세대,서울특별시 구로구 구로동 111,대,241.3,제3종일반주거지역,지정되지않음,주상용,평지,가로장방,중로각지
40,서울특별시 구로구 구로동,112-7,112,7,늘푸른 나래빌,29.93,17.39,20500,3,2016.0,2019-12-27,684.931507,연립다세대,서울특별시 구로구 구로동 112-7,대,183.1,제2종일반주거지역,지정되지않음,단독,평지,세로장방,중로한면
41,서울특별시 구로구 구로동,112-7,112,7,늘푸른 나래빌,57.80,33.58,36000,5,2016.0,2020-05-16,622.837370,연립다세대,서울특별시 구로구 구로동 112-7,대,183.1,제2종일반주거지역,지정되지않음,단독,평지,세로장방,중로한면


In [24]:
yunrip_target_df['지번주소'].nunique()

17

In [25]:
yunrip_target_df['지번주소'].unique()

array(['서울특별시 구로구 구로동 33', '서울특별시 구로구 구로동 111', '서울특별시 구로구 구로동 112-7',
       '서울특별시 구로구 구로동 252-48', '서울특별시 구로구 구로동 415-44',
       '서울특별시 구로구 구로동 448-77', '서울특별시 구로구 구로동 469-5',
       '서울특별시 구로구 구로동 470-6', '서울특별시 구로구 구로동 487-2', '서울특별시 구로구 구로동 557',
       '서울특별시 구로구 구로동 570-55', '서울특별시 구로구 구로동 685-203',
       '서울특별시 구로구 구로동 685-480', '서울특별시 구로구 구로동 715-5',
       '서울특별시 구로구 구로동 742-16', '서울특별시 구로구 구로동 798-30',
       '서울특별시 구로구 구로동 798-34'], dtype=object)

In [26]:
yunrip_target_df

,시군구,번지,본번,부번,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),부동산유형,지번주소,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
6,서울특별시 구로구 구로동,33,33,0,우성빌라,55.06,70.94,39500,1,1985.0,2019-12-30,717.399201,연립다세대,서울특별시 구로구 구로동 33,대,1981.0,제2종일반주거지역,지정되지않음,연립,평지,가로장방,중로각지
7,서울특별시 구로구 구로동,33,33,0,우성빌라,55.06,70.94,40000,2,1985.0,2020-02-11,726.480203,연립다세대,서울특별시 구로구 구로동 33,대,1981.0,제2종일반주거지역,지정되지않음,연립,평지,가로장방,중로각지
35,서울특별시 구로구 구로동,111,111,0,대원파크빌,43.88,20.76,28800,4,2016.0,2020-10-13,656.335460,연립다세대,서울특별시 구로구 구로동 111,대,241.3,제3종일반주거지역,지정되지않음,주상용,평지,가로장방,중로각지
40,서울특별시 구로구 구로동,112-7,112,7,늘푸른 나래빌,29.93,17.39,20500,3,2016.0,2019-12-27,684.931507,연립다세대,서울특별시 구로구 구로동 112-7,대,183.1,제2종일반주거지역,지정되지않음,단독,평지,세로장방,중로한면
41,서울특별시 구로구 구로동,112-7,112,7,늘푸른 나래빌,57.80,33.58,36000,5,2016.0,2020-05-16,622.837370,연립다세대,서울특별시 구로구 구로동 112-7,대,183.1,제2종일반주거지역,지정되지않음,단독,평지,세로장방,중로한면
99,서울특별시 구로구 구로동,252-48,252,48,(252-48),30.63,10.48,14000,2,1985.0,2021-04-20,457.068234,연립다세대,서울특별시 구로구 구로동 252-48,대,134.0,제2종일반주거지역,지정되지않음,주상용,평지,사다리형,중로한면
156,서울특별시 구로구 구로동,415-44,415,44,미라벨,41.81,26.47,31000,4,2018.0,2020-03-05,741.449414,연립다세대,서울특별시 구로구 구로동 415-44,대,555.0,제2종일반주거지역,지정되지않음,상업기타,평지,가로장방,중로각지
157,서울특별시 구로구 구로동,415-44,415,44,미라벨,29.39,18.61,18000,3,2018.0,2020-12-11,612.453215,연립다세대,서울특별시 구로구 구로동 415-44,대,555.0,제2종일반주거지역,지정되지않음,상업기타,평지,가로장방,중로각지
211,서울특별시 구로구 구로동,448-77,448,77,계영하이츠(448-77),52.56,25.64,26700,5,2002.0,2021-06-01,507.990868,연립다세대,서울특별시 구로구 구로동 448-77,대,126.0,제2종일반주거지역,지정되지않음,다세대,평지,사다리형,중로각지
255,서울특별시 구로구 구로동,469-5,469,5,현대가우스빌(469-5),45.46,27.86,25400,4,2016.0,2020-06-28,558.732952,연립다세대,서울특별시 구로구 구로동 469-5,대,352.0,제2종일반주거지역,지정되지않음,주거기타,평지,삼각형,중로각지


In [ ]:
officetel_df = data_prep('오피스텔', '매매')
officetel_df['부동산유형'] = '오피스텔'
print(officetel_df.shape)
officetel_df.head()

In [ ]:
'''house_merge_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)
print(house_merge_df.shape)'''

In [ ]:
target_df = house_df[
    (house_df['지번주소'].str.contains(target_gu + ' ' + target_dong))
    #&(house_df['동'] == target_dong)
    &(house_df['계약날짜'] >= (date_today - pd.DateOffset(months=target_period)))
    &(house_df['건물연식'] <= target_building_age)
    #&(house_merge_df['용도지역명1'].str.contains(target_purpose))
]
print(target_df.shape)
target_df.head()

In [ ]:
sales_idx_basedir = './한국부동산원/실거래가격지수/'
filenames = [f for f in os.listdir(sales_idx_basedir) if '연립다세대' in f]

sales_idx_df = pd.read_excel(sales_idx_basedir + filenames[0], header=[10, 11])

sales_idx_cols_list = []
for i in range(sales_idx_df.shape[1]):
    if i == 0:
        sales_idx_cols_list.append(sales_idx_df.columns[i][0])
    else:
        sales_idx_cols_list.append(sales_idx_df.columns[i][0] + '_' + sales_idx_df.columns[i][1])

sales_idx_df.columns = sales_idx_cols_list

In [ ]:
def get_sales_idx(x):
    # x is a pandas timestamp
    
    df = sales_idx_df.copy()
    
    if pd.isna(x) == True:
        return x
    else:
        year = str(x.year)
        month = str(x.month)
        if len(month) == 1:
            month = '0'+month
        
        idx = df[year+'년 '+month+'월_지수'].iloc[0]
        
        return idx

In [ ]:
def get_index_applied_prices(df):
    df = df.copy()
    
    last_idx = sales_idx_df.iloc[0,-2]
    
    last_date = sales_idx_df.columns[-1].split('_')[0]
    
    last_date = last_date.split(' ')[0].replace('년','') + '-' + last_date.split(' ')[1].replace('월','') + '-01'
    
    target_upper_limit_df = df[df['계약날짜'] < pd.to_datetime(last_date)]
    target_upper_limit_df['계약날짜_실거래지수'] = target_upper_limit_df['계약날짜'].apply(get_sales_idx)
    target_upper_limit_df['지수적용단가'] = target_upper_limit_df['단가(만원/㎡)'] / target_upper_limit_df['계약날짜_실거래지수'] * last_idx
    df['지수적용단가'] = df['단가(만원/㎡)']
    df.loc[target_upper_limit_df.index, '지수적용단가'] = target_upper_limit_df['지수적용단가']
    df['평가날짜'] = df['계약날짜']
    df.loc[target_upper_limit_df.index, '평가날짜'] = pd.to_datetime(last_date[:-2] + '01')
    
    return df

In [ ]:
target_df = get_index_applied_prices(target_df)
print(target_df.shape)
target_df.head()

In [ ]:
target_df = target_df.sort_values(['지번주소', '건축년도', '계약날짜']).reset_index(drop=True)
target_df1 = target_df[['지번주소', '건물(단지)명', '건물연식', '층', '전용면적(㎡)', '계약날짜', '거래금액(만원)', '단가(만원/㎡)', '평가날짜', '지수적용단가', '부동산유형']]
target_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/{}_{}_연립다세대_오피스텔_{}/실거래가.xlsx'.format(target_gu, target_dong, date_today_record), index=False)

In [ ]:
target_df1.head()

In [ ]:
target_df1_30 = target_df1[
    (target_df1['전용면적(㎡)'] >= 28)
    &(target_df1['전용면적(㎡)'] <= 32)
]

In [ ]:
target_df1_30.to_excel('./국토교통부_실거래가_공개시스템/집값분석/{}_{}_연립다세대_오피스텔_{}/실거래가_30.xlsx'.format(target_gu, target_dong, date_today_record), index=False)

In [ ]:
font = {'size': 16}
matplotlib.rc('font', **font)
f, ax = plt.subplots(figsize=(20,10))
plt.title('최근 {}개월 {} {} 건물연식 {}년이하 연립다세대 및 오피스텔 매매 실거래 {}건의 전용면적 단가'.format(target_period, target_gu, target_dong, target_building_age, target_df.shape[0]))
plt.grid(True)
ax.xaxis.update_units(target_df['계약날짜'])
sns.regplot(x=ax.xaxis.convert_units(target_df['계약날짜']), y=target_df['단가(만원/㎡)'])
plt.savefig('./국토교통부_실거래가_공개시스템/집값분석/{}_{}_연립다세대_오피스텔_{}/산포도.png'.format(target_gu, target_dong, date_today_record))

In [ ]:
target_last_df = target_df.drop_duplicates(subset=['지번주소', '건축년도'], keep='last').reset_index(drop=True)

In [ ]:
font = {'size': 16}
matplotlib.rc('font', **font)
f, ax = plt.subplots(figsize=(20,10))
plt.title('최근 {}개월 {} {} 건물연식 {}년이하 연립다세대 및 오피스텔 매매 실거래 {}건의 전용면적 단가'.format(target_period, target_gu, target_dong, target_building_age, target_last_df.shape[0]))
plt.grid(True)
ax.xaxis.update_units(target_last_df['계약날짜'])
sns.regplot(x=ax.xaxis.convert_units(target_last_df['계약날짜']), y=target_last_df['단가(만원/㎡)'])
plt.savefig('./국토교통부_실거래가_공개시스템/집값분석/{}_{}_연립다세대_오피스텔_{}/마지막_산포도.png'.format(target_gu, target_dong, date_today_record))

In [ ]:
target_last_df.to_excel('./국토교통부_실거래가_공개시스템/집값분석/{}_{}_연립다세대_오피스텔_{}/마지막_실거래가.xlsx'.format(target_gu, target_dong, date_today_record), index=False)

In [ ]:
target_df2 = house_df[
    (house_df['지번주소'].str.contains(target_gu + ' ' + target_dong))
    #&(house_df['동'] == target_dong)
    #&(house_merge_df['계약날짜'] >= (date_today - pd.DateOffset(months=target_period)))
    &(house_df['계약날짜기준_건물연식'] <= target_building_age)
    #&(house_merge_df['용도지역명1'].str.contains(target_purpose))
]
print(target_df2.shape)
target_df2.head()

In [ ]:
target_df2 = get_index_applied_prices(target_df2)

In [ ]:
font = {'size': 12}
matplotlib.rc('font', **font)
f, ax = plt.subplots(figsize=(12,6))
plt.title('{} {} 계약날짜기준 건물연식 {}년이하 연립다세대 및 오피스텔 매매 실거래 {}건의 전용면적 단가'.format(target_gu, target_dong, target_building_age, target_df2.shape[0]))
plt.grid(True)
sns.histplot(target_df2['단가(만원/㎡)'], kde=True)

In [ ]:
target_df3 = target_df2[['지번주소', '건물(단지)명', '건축년도', '층', '전용면적(㎡)', '계약날짜', '단가(만원/㎡)', '평가날짜', '지수적용단가', '부동산유형']]
target_df3 = target_df3.sort_values(['지번주소', '건축년도', '계약날짜']).reset_index(drop=True)
target_df3.to_excel('./국토교통부_실거래가_공개시스템/집값분석/{}_{}_연립다세대_오피스텔_{}/지수적용_실거래가.xlsx'.format(target_gu, target_dong, date_today_record), index=False)

In [ ]:
target_df3_30 = target_df3[
    (target_df3['전용면적(㎡)'] >= 28)
    &(target_df3['전용면적(㎡)'] <= 32)
]

In [ ]:
target_df3_30.to_excel('./국토교통부_실거래가_공개시스템/집값분석/{}_{}_연립다세대_오피스텔_{}/지수적용_실거래가_30.xlsx'.format(target_gu, target_dong, date_today_record), index=False)

In [ ]:
concat_df = pd.concat([target_df2])

In [ ]:
concat_df = concat_df.sort_values(['지번주소', '건축년도', '전용면적(㎡)', '계약날짜']).reset_index(drop=True)
concat_df = concat_df.drop_duplicates(['지번주소', '건축년도', '전용면적(㎡)'], keep='last').reset_index(drop=True)

In [ ]:
concat_df['단가평균'] = concat_df[['지번주소', '건축년도', '단가(만원/㎡)']].groupby(['지번주소', '건축년도']).transform('mean')['단가(만원/㎡)']
concat_df['지수적용단가평균'] = concat_df[['지번주소', '건축년도', '지수적용단가']].groupby(['지번주소', '건축년도']).transform('mean')['지수적용단가']

In [ ]:
concat_df = concat_df.sort_values(['지번주소', '건축년도', '평가날짜']).reset_index(drop=True)
print(concat_df.shape)
concat_df.head()

In [ ]:
concat_last_df = concat_df.drop_duplicates(subset=['지번주소', '건축년도'], keep='last')
print(concat_last_df.shape)
concat_last_df

In [ ]:
font = {'size': 16}
matplotlib.rc('font', **font)
f, ax = plt.subplots(figsize=(20,10))
plt.title('{} {} 계약날짜기준 건물연식 {}년이하 연립다세대 및 오피스텔 매매 실거래 지번별 {}곳의 전용면적 실거래가격지수 적용후 단가평균'.format(target_gu, target_dong, target_building_age, concat_last_df.shape[0]))
plt.grid(True)
sns.histplot(concat_last_df['지수적용단가'], kde=True)
plt.savefig('./국토교통부_실거래가_공개시스템/집값분석/{}_{}_연립다세대_오피스텔_{}/지번별_분포도.png'.format(target_gu, target_dong, date_today_record))

In [ ]:
concat_last_df = concat_last_df.sort_values(['지번주소']).reset_index(drop=True)

In [ ]:
concat_last_df = concat_last_df[['지번주소', '건물(단지)명', '건축년도', '전용면적(㎡)', '층', '계약날짜', '단가(만원/㎡)', '평가날짜', '지수적용단가', '부동산유형']]
concat_last_df.rename(columns={'계약날짜':'마지막_계약날짜'}, inplace=True)
concat_last_df.head()

In [ ]:
concat_last_df.to_excel('./국토교통부_실거래가_공개시스템/집값분석/{}_{}_연립다세대_오피스텔_{}/지수적용_마지막_실거래가.xlsx'.format(target_gu, target_dong, date_today_record), index=False)